In [4]:
import requests
file_url = "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
	
r = requests.get(file_url, stream = True)

with open("/content/drive/MyDrive/Python_Panda/k.zip", "wb") as file:
	for block in r.iter_content(chunk_size = 1024):
		if block:
			file.write(block)


In [6]:
!unzip "/content/drive/MyDrive/Python_Panda/k.zip"

Archive:  /content/drive/MyDrive/Python_Panda/k.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       
Archive:  /content/drive/MyDrive/Python_Panda/k.zip
replace ml-25m/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/tags.csv         
replace ml-25m/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/links.csv        
replace ml-25m/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/README.txt       
replace ml-25m/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/ratings.csv      y
y
y

replace ml-25m/genome-tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: ml-25m/genome-tags.csv  
replace ml-25m/genome-scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   i

In [83]:
import pandas as pd
movies = pd.read_csv("/content/ml-25m/movies.csv")

In [84]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [85]:
import re
def clean_title(title):
  return re.sub("[^a-zA-Z0-9 ]","", title)

In [86]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [87]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["clean_title"])

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
  title = clean_title(title)
  query_vec = vectorizer.transform([title])
  similarity = cosine_similarity(query_vec, tfidf).flatten()
  indices = np.argpartition(similarity, -5)[-5:]
  results = movies.iloc[indices][::-1]
  return results

In [90]:
from tables import description
import ipywidgets as widgets
from IPython.display import display
movie_input = widgets.Text(
    value="Toy Store",
    description="Movie Title:",
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
  with movie_list:
    movie_list.clear_output()
    title = data["new"]
    if len(title) > 5:
       display(search(title))


movie_input.observe(on_type, names='value')

display(movie_input, movie_list)

Text(value='Toy Store', description='Movie Title:')

Output()

In [91]:
ratings = pd.read_csv("/content/ml-25m/ratings.csv")

In [92]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [93]:
movie_id = 1

In [94]:
similar_users = ratings[(ratings["movieId"] == movie_id)& (ratings["rating"] >= 4)]["userId"].unique()

In [95]:
similar_users

array([     3,      5,      8, ..., 162530, 162533, 162534])

In [96]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users) & (ratings["rating"] >= 4) )]

In [97]:
similar_user_recs

,userId,movieId,rating,timestamp
254,3,1,4.0,1439472215
255,3,29,4.5,1484754967
256,3,32,4.5,1439474635
257,3,50,5.0,1439474391
258,3,111,4.0,1484753849
...,...,...,...,...
24999332,162534,166643,4.0,1526891765
24999342,162534,171763,4.0,1526717390
24999348,162534,177593,5.0,1526666314
24999351,162534,177765,4.0,1526666311


In [98]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [99]:
similar_user_recs

Series([], dtype: float64)

In [76]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index) & (ratings["rating"] > 4) )]

In [77]:
all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [78]:
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [79]:
rec_percentages

,,,,similar,all


In [80]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [81]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [100]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

ValueError: ignored

In [101]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [102]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()